# VAE를 이용한 생성 모델
- 새로운 분자의 SMILES를 출력
- MolrculeNet이 제공하는 SMILES 데이터셋 MUV 사용 (약 90000개 제공)
 - Maximum Unbiased Validation(MUV) - 17개의 태스크 포함

In [ ]:
!pip install DeepChem

     |████████████████████████████████| 608 kB 5.2 MB/s 
     |████████████████████████████████| 22.7 MB 1.6 MB/s 


In [ ]:
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'retina'

In [ ]:
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids

# SMILES 문자열의 규칙을 파악: 문자(토큰)의 목록, 문자열의 최대길이 등

tokens = set()
for s in train_smiles:
  tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

# 모델 만들기
# AspuruGuzikAutoEncoder: 인코더는 합성곱신경망을, 디코더는 순환신경망을 사용
# 학습속도를 조절하기 위해서 ExponentialDecay를 사용한다
# 0.001에서 시작하고 이포크마다 0.95배로 감소시킨다

from deepchem.models.optimizers import ExponentialDecay
from deepchem.models.seqtoseq import AspuruGuzikAutoEncoder
batch_size = 100
batches_per_epoch = len(train_smiles)/batch_size
learning_rate = ExponentialDecay(0.001, 0.95, batches_per_epoch)
model = AspuruGuzikAutoEncoder(tokens, max_length, model_dir='vae', batch_size=batch_size, learning_rate=learning_rate)

# 시퀀스 생성 함수 정의

def generate_sequences(epochs):
  for i in range(epochs):
    for s in train_smiles:
      yield (s, s)

# AspuruGuzikAutoEncoder이 제공하는 자체 학습 함수 (이포크수 지정)

model.fit_sequences(generate_sequences(2)) # 50 이포크 수

# 학습된 모델을 이용하여 새로운 분자를 만든다
# 모델에 들어가는 벡터의 크기를 지정한다 (예: 196)
# 벡터를 1000개 생성하겠다
# 생성된 분자들중 유효한 SMILES를 걸러내기 위해서 RDKit의 MolFromSmiles를 사용한다

from rdkit import Chem
predictions = model.predict_from_embeddings(np.random.normal(size=(1000,196)))
molecules = []
for p in predictions:
  smiles = ''.join(p)
  if Chem.MolFromSmiles(smiles) is not None:
    molecules.append(smiles)
print()
print('Generated molecules:')
for m in molecules:
  print(m)



Generated molecules:
CCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCCCC
CC1CCCCCCCCCCCCCCCCCCCCCC1
C=CCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCCCCCCCCCC
C=CCCCCCCCCCCCCCCCCCCC

In [ ]:
molecules

['CCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CC1CCCCCCCCCCCCCCCCCCCCCC1',
 'C=CCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'C=CCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCC

## 생성된 SMILES 들에 대해서 유효하지 않거나 약물로서 가치가 없는 분자를 걸러내야 한다
- 제거하고 싶은 분자가 있는지 찾는다
- MolFromSmiles()을 사용해 SMILES 문자열들을 분자 객체로 변환한다
- 분자의 크기를 확인한다 (10보다 작으면 상호작용에 필요한 에너지가 불충분하고, 50 이상이면 분자의 용해도가 너무 낮아 문제가 된다)
- 수소를 제외한 분자의 크기를 GetNumAtoms()로 얻는다
- 약물과 얼마나 유사한지를 판단하기 위해서 QED(Quantitave Estimate of Drugness)를 많이 사용한다
 - QED: 계산된 속성 집합과 판매된 약물의 동일한 특성 분포를 정량화 한 것 (Richard Bickerton 이 제안)
 - 1에 가까울수록 기존의 약물과 유사하다고 본다
 - QED > 0.5 인 분자만 고른 후 결과를 시각화 한다

In [ ]:
from rdkit import Chem
molecules_new = [Chem.MolFromSmiles(x) for x in molecules]
# moleculs_new = [Chem.MolFromSmiles(x) for x in smlies_list]
print(sorted(x.GetNumAtoms() for x in molecules_new))

[16, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 35]


In [ ]:
good_mol_list = [x for x in molecules_new if x.GetNumAtoms() > 10 and x.GetNumAtoms() < 50]
print(len(good_mol_list))

224


In [ ]:
from rdkit.Chem import QED
qed_list = [QED.qed(x) for x in good_mol_list]

In [ ]:
qed_list

[0.1826494365002578,
 0.14453274779218342,
 0.16993743727121963,
 0.16993743727121963,
 0.1576690191819601,
 0.12484569683941008,
 0.13497266961693322,
 0.12475265989203768,
 0.1826494365002578,
 0.17633918766608803,
 0.19567989082850498,
 0.09887211885056306,
 0.17633918766608803,
 0.1459754656495484,
 0.10694445345920522,
 0.41320041499253696,
 0.1653911940322381,
 0.1826494365002578,
 0.11562669846841413,
 0.19567989082850498,
 0.19567989082850498,
 0.1459754656495484,
 0.13451251416028162,
 0.17633918766608803,
 0.11562669846841413,
 0.12484569683941008,
 0.15483007424381995,
 0.1459754656495484,
 0.11538186127227373,
 0.11562669846841413,
 0.12475265989203768,
 0.09265803311375906,
 0.1576690191819601,
 0.1069018002662766,
 0.09887211885056306,
 0.11562669846841413,
 0.11538186127227373,
 0.09887211885056306,
 0.11538186127227373,
 0.1459754656495484,
 0.1826494365002578,
 0.13497266961693322,
 0.15483007424381995,
 0.09887211885056306,
 0.11562669846841413,
 0.13451251416028162,


In [ ]:
final_mol_list = [(a,b) for a,b in zip(good_mol_list, qed_list) if b > 0.3] # 0.5를 낮춤
final_mol_list

[(<rdkit.Chem.rdchem.Mol at 0x7f59da512bc0>, 0.41320041499253696),
 (<rdkit.Chem.rdchem.Mol at 0x7f59da4fbcb0>, 0.4007624160001301)]

In [ ]:
from rdkit.Chem import Draw
Draw.MolsToGridImage(
    [x[0] for x  in final_mol_list],
    molsPerRow = 3, useSVG=True,
    subImgSize = (250, 250),
    returnPNG=True,
    legends=[f"{x[1]:.2f}" for x in final_mol_list])


"<?xml version='1.0' encoding='iso-8859-1'?>\n<svg version='1.1' baseProfile='full'\n              xmlns='http://www.w3.org/2000/svg'\n                      xmlns:rdkit='http://www.rdkit.org/xml'\n                      xmlns:xlink='http://www.w3.org/1999/xlink'\n                  xml:space='preserve'\nwidth='750px' height='250px' viewBox='0 0 750 250'>\n<!-- END OF HEADER -->\n<rect style='opacity:1.0;fill:#FFFFFF;stroke:none' width='750.0' height='250.0' x='0.0' y='0.0'> </rect>\n<path class='bond-0 atom-0 atom-1' d='M 229.3,112.5 L 204.5,112.5' style='fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:2.0px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<path class='bond-1 atom-1 atom-2' d='M 204.5,112.5 L 201.8,92.4' style='fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:2.0px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<path class='bond-2 atom-2 atom-3' d='M 199.3,83.5 L 193.2,69.6' style='fill:none;fill-rule:evenodd;stroke:#000000;

참고:
https://www.rdkit.org/docs/GettingStartedInPython.html#drawing-molecules

In [ ]:
img=Draw.MolsToGridImage(subms,molsPerRow=4,subImgSize=(200,200),
                         legends=[x.GetProp("_Name") for x in subms])    
img.save('images/cdk2_molgrid.aligned.o.png')

In [ ]:
final_mol_list

[]